In [66]:
#####Monthly Long Base Case - Current Month - POE#####


import pandas as pd
import numpy as np
import json
import datetime as dt
from datetime import datetime, date, time,timedelta
import pandas as pd
from csv import DictWriter
import urllib
import re,datetime
import os
import time
from os import walk
import re
import  gc
from pyxirr import xirr


trigger = [100,25,50,75,125,150, 175, 200]

delta_list = [15,20,25,30,35,40,45,50]

list1=[100,25,50,125,150,175,200]
N1=[1]#,1,2]#3,1,2]#,3,4,0]
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")
leverage=6
for delta in tqdm(delta_list):
    for nnn in tqdm(N1):
                plvalue = []
                pl = 0
                transaction_cost = 0
                contract_value=0
            # for t in trigger:
                output_path=r'C:\Projects\Straddles\NEW\2021-2023/Monthly Base Case/'+str(delta)+'_delta'+'/'+str(nnn)
                df=pd.read_csv(output_path+'/t3-w3-3.csv')

                df['lotsize']=1
                df['qty']=1

                df['Exposure']=df['EQ_Close_CE']*2*df['lotsize']
                df['Margin']=df['Exposure']/leverage


                initial_equity=10000000
                poe=0.01
                equity=poe*initial_equity                # df['units'] = (initial_equity/df['Margin']).astype(int)
                # df['qty_final'] = df["qty"]*df['units']

                df.to_csv(output_path+"/unit check1=w3.csv")


                df=pd.read_csv(output_path+"/unit check1=w3.csv")
                # print(6,nnn1)
                temp=0
                for idi, row in df.iterrows():
                    
                        if nnn>0:
                            if row["Marker"] == "F":
                                df.loc[idi,'CE_Points']=df.loc[idi,'PE_Points']=df.loc[idi,'Points']=0
                                
                            # if row["Marker"] == "F":
                                df.loc[idi, 'pl_check'] = (pl+initial_equity)
                                df.loc[idi+1:, 'pl_check'] = np.nan

                                (df.loc[idi ,'available_margin']) = (df.loc[idi ,'pl_check'])*leverage

                                df.loc[idi , 'units_new'] = (((df.loc[idi ,'available_margin']))/df.loc[idi, 'EQ_Close_CE'])
                                df.loc[idi+1: , 'units_new'] = np.nan

                                df.loc[idi,  'qty_final'] = df.loc[idi,"qty"] * df.loc[idi, 'units_new']
                                df.loc[idi+1:, 'qty_final'] = np.nan
                                df['qty_final']=df['qty_final'].ffill()
                                # df.loc[idi, 'plvalue_' + index] = 0
                                # df.loc[idi, 'transaction_cost'] = 0
                                contract_value = df.loc[idi, "qty_final"]*(df.loc[idi,  "OPT_Close_CE"]+df.loc[idi,  "OPT_Close_PE"])
                                pl=pl+df.loc[idi,'Points']*df.loc[idi, "qty_final"]
                                df.loc[idi,'Points_final']=df.loc[idi,'Points']*df.loc[idi,'qty_final']

                            elif row['Marker']=='L':
                                df.loc[idi,'CE_Points']=df.loc[idi,  "OPT_Close_CE"]-df.loc[idi-1,  "OPT_Close_CE"]
                                df.loc[idi,'PE_Points']=df.loc[idi,  "OPT_Close_PE"]-df.loc[idi-1,  "OPT_Close_PE"]
                                df.loc[idi,'Points']=(df.loc[idi,  "OPT_Close_PE"]-df.loc[idi-1,  "OPT_Close_PE"])+(df.loc[idi,  "OPT_Close_CE"]-df.loc[idi-1,  "OPT_Close_CE"])
                                df.loc[idi,'Points_final']=df.loc[idi,'Points']*df.loc[idi,'qty_final']
                                contract_value += df.loc[idi, "qty_final"]*(df.loc[idi,  "OPT_Close_CE"]+df.loc[idi,  "OPT_Close_PE"])
                                df.loc[idi,'transaction_cost']=contract_value*0.01
                                pl=pl+df.loc[idi,'Points']*df.loc[idi,  'qty_final']-df.loc[idi,'transaction_cost']

                            elif row['Marker']!='F' or row['Marker']!='L':
                                df.loc[idi,'CE_Points']=df.loc[idi,  "OPT_Close_CE"]-df.loc[idi-1,  "OPT_Close_CE"]
                                df.loc[idi,'PE_Points']=df.loc[idi,  "OPT_Close_PE"]-df.loc[idi-1,  "OPT_Close_PE"]
                                df.loc[idi,'Points']=(df.loc[idi,  "OPT_Close_PE"]-df.loc[idi-1,  "OPT_Close_PE"])+(df.loc[idi,  "OPT_Close_CE"]-df.loc[idi-1,  "OPT_Close_CE"])
                                pl=pl+df.loc[idi,'Points']*df.loc[idi,  'qty_final']
                                df.loc[idi,'Points_final']=df.loc[idi,'Points']*df.loc[idi,'qty_final']

                                

                # df['Points_final']=df['Points']*df['qty_final']
                df['Total_points']=df['Points_final'].cumsum()
                # print(7,nnn1)
                df.to_csv(output_path+'/pl_points_'+str(delta)+'.csv')

                df=pd.read_csv(output_path+'/pl_points_'+str(delta)+'.csv')
                temp=0
                df['Total_points']=df['Points_final'].cumsum()
                df['eq_curve']=df['Points_final']
                df.loc[0,'eq_curve']=initial_equity
                df['eq_curve']=df['eq_curve'].cumsum()
                output_path1=r'C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly base case/'+str(nnn)+'/'
                if not os.path.exists(output_path1):
                    os.makedirs(output_path1)
                df.to_csv(output_path1+'/eq_curve_'+str(delta)+'.csv')
                print((output_path1+'/eq_curve_'+str(delta)+'.csv'))
                print('Done')
                # xxx
                # print(7,nnn1)
                

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly base case/1//eq_curve_15.csv
Done


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly base case/1//eq_curve_20.csv
Done


...

C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly base case/1//eq_curve_40.csv
Done


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly base case/1//eq_curve_45.csv
Done


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly base case/1//eq_curve_50.csv
Done


In [68]:
#####Monthly Long Adjustments - Current Month - POE#####


trigger = [100,25,50,75,125,150, 175, 200]

delta_list=['50','45', '40', '35', '30', '25','20','15']

list1=[100,25,50,125,150,175,200]
N1=[1]#,1,2]#3,1,2]#,3,4,0]
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")
leverage=6
for delta in tqdm(delta_list):
    for nnn in tqdm(N1):
            plvalue = []
            pl = 0
            transaction_cost = 0
            contract_value=0
            for t in trigger:
                plvalue = []
                pl = 0
                transaction_cost = 0
                contract_value=0
                output_path=r'C:\Projects\Straddles\NEW\2021-2023/Monthly adj/'+delta+'_delta'+'/'+str(nnn)+'/'+str(t)
                df=pd.read_csv(output_path+'/t3-w3-3-2.csv')
                poe=0.01
                df['lotsize']=1
                df['qty']=1

                df['Exposure']=df['EQ_Close_CE']*2*df['lotsize']
                df['Margin']=df['Exposure']/leverage


                initial_equity=10000000
                equity=poe*initial_equity
                # df['units'] = (initial_equity/df['Margin']).astype(int)
                # df['qty_final'] = df["qty"]*df['units']

                df.to_csv(output_path+"/unit check1=w3.csv")


                df=pd.read_csv(output_path+"/unit check1=w3.csv")
                # print(6,nnn1)
                temp=0
                if nnn==4:
                    for idi, row in df.iterrows():
                        
                        if idi==0:
                            df.loc[idi,'CE_Points']=df.loc[idi,'PE_Points']=df.loc[idi,'Points']=0
                            pl=pl+df.loc[idi,'Points']
                        # if row["Marker"] == "F":
                            df.loc[idi, 'pl_check'] = (pl+initial_equity)
                            df.loc[idi+1:, 'pl_check'] = np.nan
                            
                            (df.loc[idi ,'available_margin']) = (df.loc[idi ,'pl_check'])*leverage

                            df.loc[idi , 'units_new'] = (((df.loc[idi ,'available_margin']))/df.loc[idi, 'EQ_Close_CE'])
                            df.loc[idi+1: , 'units_new'] = np.nan

                            df.loc[idi,  'qty_final'] = df.loc[idi,"qty"] * df.loc[idi, 'units_new']
                            df.loc[idi+1:, 'qty_final'] = np.nan
                            df['qty_final']=df['qty_final'].ffill()
                            # df.loc[idi, 'plvalue_' + index] = 0
                            # df.loc[idi, 'transaction_cost'] = 0
                            contract_value = df.loc[idi, "qty_final"]*(df.loc[idi,  "OPT_Close_CE"]+df.loc[idi,  "OPT_Close_PE"])
                            continue
                        if row["Marker"] != "F":
                            df.loc[idi,'CE_Points']=-df.loc[idi,  "OPT_Close_CE"]+df.loc[idi-1,  "OPT_Close_CE"]
                            df.loc[idi,'PE_Points']=-df.loc[idi,  "OPT_Close_PE"]+df.loc[idi-1,  "OPT_Close_PE"]
                            df.loc[idi,'Points']=(-df.loc[idi,  "OPT_Close_PE"]+df.loc[idi-1,  "OPT_Close_PE"])+(-df.loc[idi,  "OPT_Close_CE"]+df.loc[idi-1,  "OPT_Close_CE"])
                            pl=pl+df.loc[idi,'Points']
                            
                        if row["Marker"] == "F":
                            df.loc[idi,'CE_Points']=df.loc[idi,'PE_Points']=df.loc[idi,'Points']=0
                            pl=pl+df.loc[idi,'Points']
                        # if row["Marker"] == "F":
                            df.loc[idi, 'pl_check'] = (pl+ initial_equity)
                            df.loc[idi+1:, 'pl_check'] = np.nan
                            
                            (df.loc[idi ,'available_margin']) = (df.loc[idi ,'pl_check'])*leverage

                            df.loc[idi , 'units_new'] = (((df.loc[idi ,'available_margin']))/df.loc[idi, 'EQ_Close_CE'])
                            df.loc[idi+1: , 'units_new'] = np.nan

                            df.loc[idi,  'qty_final'] = df.loc[idi,"qty"] * df.loc[idi, 'units_new']
                            df.loc[idi+1:, 'qty_final'] = np.nan
                            df['qty_final']=df['qty_final'].ffill()
                            # df.loc[idi, 'plvalue_' + index] = 0
                            # df.loc[idi, 'transaction_cost'] = 0
                            contract_value = df.loc[idi, "qty_final"]*(df.loc[idi,  "OPT_Close_CE"]+df.loc[idi,  "OPT_Close_PE"])
                        if row['Marker']=='L':
                            contract_value += df.loc[idi, "qty_final"]*(df.loc[idi,  "OPT_Close_CE"]+df.loc[idi,  "OPT_Close_PE"])
                            df.loc[idi,'transaction_cost']=contract_value*0.01

                else:       
                    for idi, row in df.iterrows():
                        
                        if idi==0:
                            df.loc[idi,'CE_Points']=df.loc[idi,'PE_Points']=df.loc[idi,'Points']=0
                            pl=pl+df.loc[idi,'Points']
                        # if row["Marker"] == "F":
                            df.loc[idi, 'pl_check'] = (pl+initial_equity)
                            df.loc[idi+1:, 'pl_check'] = np.nan
                            
                            (df.loc[idi ,'available_margin']) = (df.loc[idi ,'pl_check'])*leverage

                            df.loc[idi , 'units_new'] = (((df.loc[idi ,'available_margin']))/df.loc[idi, 'EQ_Close_CE'])
                            df.loc[idi+1: , 'units_new'] = np.nan

                            df.loc[idi,  'qty_final'] = df.loc[idi,"qty"] * df.loc[idi, 'units_new']
                            df.loc[idi+1:, 'qty_final'] = np.nan
                            df['qty_final']=df['qty_final'].ffill()
                            # df.loc[idi, 'plvalue_' + index] = 0
                            # df.loc[idi, 'transaction_cost'] = 0
                            contract_value = df.loc[idi, "qty_final"]*(df.loc[idi,  "OPT_Close_CE"]+df.loc[idi,  "OPT_Close_PE"])
                            continue

                        if row['Marker2']=='L':# and row['Marker']!='L':
                            df.loc[idi,'CE_Points']=0 #-df.loc[idi,  "OPT_Close_CE"]+df.loc[idi-1,  "OPT_Close_CE"]
                            df.loc[idi,'PE_Points']=0 #-df.loc[idi,  "OPT_Close_PE"]+df.loc[idi-1,  "OPT_Close_PE"]
                            df.loc[idi,'Points']=0 #(-df.loc[idi,  "OPT_Close_PE"]+df.loc[idi-1,  "OPT_Close_PE"])+(-df.loc[idi,  "OPT_Close_CE"]+df.loc[idi-1,  "OPT_Close_CE"])

                        else:
                            if nnn>0:
                                # if row['Marker3']=='F' and row['Status1']==1:
                                #     df.loc[idi,'CE_Points']=0 #-df.loc[idi,  "OPT_Close_CE"]+df.loc[idi-1,  "OPT_Close_CE"]
                                #     df.loc[idi,'PE_Points']=0 #-df.loc[idi,  "OPT_Close_PE"]+df.loc[idi-1,  "OPT_Close_PE"]
                                #     df.loc[idi,'Points']=0
                                # else:
                                if row["Marker"] == "F":
                                    df.loc[idi,'CE_Points']=df.loc[idi,'PE_Points']=df.loc[idi,'Points']=0
                                    contract_value= df.loc[idi, "qty_final"]*(df.loc[idi,  "OPT_Close_CE"]+df.loc[idi,  "OPT_Close_PE"])
                                    
                                    
                                # if row["Marker"] == "F":
                                    df.loc[idi, 'pl_check'] = (pl+initial_equity)
                                    df.loc[idi+1:, 'pl_check'] = np.nan

                                    (df.loc[idi ,'available_margin']) = (df.loc[idi ,'pl_check'])*leverage

                                    df.loc[idi , 'units_new'] = (((df.loc[idi ,'available_margin']))/df.loc[idi, 'EQ_Close_CE'])
                                    df.loc[idi+1: , 'units_new'] = np.nan

                                    df.loc[idi,  'qty_final'] = df.loc[idi,"qty"] * df.loc[idi, 'units_new']
                                    df.loc[idi+1:, 'qty_final'] = np.nan
                                    df['qty_final']=df['qty_final'].ffill()
                                    df.loc[idi,'Points_final']=df.loc[idi,'Points']*df.loc[idi,'qty_final']
                                    pl=pl+df.loc[idi,'Points_final']
                            # df.loc[idi, 'plvalue_' + index] = 0
                            # df.loc[idi, 'transaction_cost'] = 0

                                elif df.loc[idi-1,'Status1']==1:
                                    df.loc[idi,'CE_Points']=df.loc[idi,  "OPT_Close_CE"]-df.loc[idi-1,  "new_OPT_Close_CE"]
                                    df.loc[idi,'PE_Points']=df.loc[idi,  "OPT_Close_PE"]-df.loc[idi-1,  "new_OPT_Close_PE"]
                                    df.loc[idi,'Points']=df.loc[idi,'CE_Points']+df.loc[idi,'PE_Points']
                                    
                                    contract_value= df.loc[idi, "qty_final"]*(df.loc[idi-1,  "new_OPT_Close_CE"]+df.loc[idi-1,  "new_OPT_Close_PE"])
                                # if row["Marker"] == "F":
                                    df.loc[idi, 'pl_check'] = (pl+ initial_equity)
                                    df.loc[idi+1:, 'pl_check'] = np.nan

                                    (df.loc[idi ,'available_margin']) = (df.loc[idi ,'pl_check'])*leverage

                                    df.loc[idi , 'units_new'] = (((df.loc[idi ,'available_margin']))/df.loc[idi, 'EQ_Close_CE'])
                                    df.loc[idi+1: , 'units_new'] = np.nan

                                    df.loc[idi,  'qty_final'] = df.loc[idi,"qty"] * df.loc[idi, 'units_new']
                                    df.loc[idi+1:, 'qty_final'] = np.nan
                                    df['qty_final']=df['qty_final'].ffill()
                                    df.loc[idi,'Points_final']=df.loc[idi,'Points']*df.loc[idi,'qty_final']
                                    pl=pl+df.loc[idi,'Points_final']
                                    
                                else:
                                    df.loc[idi,'CE_Points']=df.loc[idi,  "OPT_Close_CE"]-df.loc[idi-1,  "OPT_Close_CE"]
                                    df.loc[idi,'PE_Points']=df.loc[idi,  "OPT_Close_PE"]-df.loc[idi-1,  "OPT_Close_PE"]
                                    df.loc[idi,'Points']=(df.loc[idi,  "OPT_Close_PE"]-df.loc[idi-1,  "OPT_Close_PE"])+(df.loc[idi,  "OPT_Close_CE"]-df.loc[idi-1,  "OPT_Close_CE"])
                                    df.loc[idi,'Points_final']=df.loc[idi,'Points']*df.loc[idi,'qty_final']
                                    pl=pl+df.loc[idi,'Points_final']
                                                                        
                                if row['Marker']=='L' or df.loc[idi,'Status1']==1:
                                    contract_value+= df.loc[idi, "qty_final"]*(df.loc[idi,  "OPT_Close_CE"]+df.loc[idi,  "OPT_Close_PE"])
                                    df.loc[idi,'transaction_cost']=contract_value*0.01
                                    pl=pl-df.loc[idi,'transaction_cost']
                                    df.loc[idi,'Points_final']=df.loc[idi,'Points']*df.loc[idi,'qty_final']-df.loc[idi,'transaction_cost']

                                                                
                                                                    
#                                 if row['Marker']=='F' or df.loc[idi,'Status1']==1:
#                                     df.loc[idi,'CE_Points']=df.loc[idi,'PE_Points']=df.loc[idi,'Points']=0
#                                     pl=pl+df.loc[idi,'Points']
#                                 # if row["Marker"] == "F":
#                                     df.loc[idi, 'pl_check'] = pl + initial_equity
#                                     df.loc[idi+1:, 'pl_check'] = np.nan
#                                     # df.loc[idi,'CE_Points']=df.loc[idi,'PE_Points']=df.loc[idi,'Points']=0

#                                     (df.loc[idi ,'available_margin']) = (df.loc[idi ,'pl_check'])

#                                     df.loc[idi , 'units_new'] = (((df.loc[idi ,'pl_check']))/df.loc[idi, 'Margin'])
#                                     df.loc[idi+1: , 'units_new'] = np.nan

#                                     df.loc[idi,  'qty_final'] = df.loc[idi,"qty"] * df.loc[idi, 'units_new']
#                                     df.loc[idi+1:, 'qty_final'] = np.nan
#                                     df['qty_final']=df['qty_final'].ffill()
#                                     # df.loc[idi, 'plvalue_' + index] = 0
#                                     contract_value = df.loc[idi, "qty_final"]*(df.loc[idi,  "OPT_Close_CE"]+df.loc[idi,  "OPT_Close_PE"])
#                                     if df.loc[idi-1,'Status1']==1:
#                                         contract_value = df.loc[idi, "qty_final"]*(df.loc[idi-1,  "new_OPT_Close_CE"]+df.loc[idi-1,  "new_OPT_Close_PE"])
                                

                df['Points_final_2']=df['Points']*df['qty_final']
                df['Total_points']=df['Points_final'].cumsum()
                # print(7,nnn1)
                df.to_csv(output_path+'/pl_points_'+str(delta)+'.csv')

                df=pd.read_csv(output_path+'/pl_points_'+str(delta)+'.csv')
                temp=0
                df['Total_points']=df['Points_final'].cumsum()
                df['eq_curve']=df['Points_final']
                df.loc[0,'eq_curve']=initial_equity
                df['eq_curve']=df['eq_curve'].cumsum()
                output_path1=r'C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/'+str(nnn)+'/'+str(t)
                if not os.path.exists(output_path1):
                    os.makedirs(output_path1)
                df.to_csv(output_path1+'/eq_curve_'+str(delta)+'_'+str(t)+'.csv')
                print((output_path1+'/eq_curve_'+str(delta)+'_'+str(t)+'.csv'))
                print('Done')
                
                # print(7,nnn1)
                

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/100/eq_curve_50_100.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/25/eq_curve_50_25.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/50/eq_curve_50_50.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/75/eq_curve_50_75.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/125/eq_curve_50_125.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/150/eq_curve_50_150.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/175/eq_curve_50_175.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/200/eq_curve_50_200.csv
Done


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/100/eq_curve_45_100.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/25/eq_curve_45_25.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/50/eq_curve_45_50.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/75/eq_curve_45_75.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/125/eq_curve_45_125.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/150/eq_curve_45_150.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/175/eq_curve_45_175.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/200/eq_curve_45_200.csv
Done


...

C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/100/eq_curve_25_100.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/25/eq_curve_25_25.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/50/eq_curve_25_50.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/75/eq_curve_25_75.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/125/eq_curve_25_125.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/150/eq_curve_25_150.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/175/eq_curve_25_175.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/200/eq_curve_25_200.csv
Done


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/100/eq_curve_20_100.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/25/eq_curve_20_25.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/50/eq_curve_20_50.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/75/eq_curve_20_75.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/125/eq_curve_20_125.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/150/eq_curve_20_150.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/175/eq_curve_20_175.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/200/eq_curve_20_200.csv
Done


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/100/eq_curve_15_100.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/25/eq_curve_15_25.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/50/eq_curve_15_50.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/75/eq_curve_15_75.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/125/eq_curve_15_125.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/150/eq_curve_15_150.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/175/eq_curve_15_175.csv
Done
C:\Projects\Straddles\NEW\2021-2023/poe-6xlev/Monthly Adj/1/200/eq_curve_15_200.csv
Done


In [71]:
#####Weekly Short Adjustments - Current Week - POE#####


import pandas as pd
import numpy as np
import json
import datetime as dt
from datetime import datetime, date, time,timedelta
import pandas as pd
from csv import DictWriter
import urllib
import re,datetime
import os
import time
from os import walk
import re
import  gc
from pyxirr import xirr

delta_list=['50','45', '40', '35', '30', '25','20','15']

trigger = [100,25,50,75,125,150, 175, 200]
trigger1=[100]
list1=[100,25,50,125,150,175,200]
N1=[1,2,3]#3,1,2]#,3,4,0]
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")
leverage=6
for delta in tqdm(delta_list):
    for nnn in tqdm(N1):
            plvalue = []
            pl = 0
            transaction_cost = 0
            contract_value=0
            for t in trigger:
                plvalue = []
                pl = 0
                transaction_cost = 0
                contract_value=0
                output_path=r'C:\Projects\Straddles\NEW\2021-2023/Weekly Adj'+'/'+delta+'_delta'+'/'+str(nnn)+'/'+str(t)
                #output_path=r'D:\Straddles BAcktest Files\NEW\N_days'+'/'+str(nnn)+'/'+delta+'_delta'
                #df=pd.read_csv(output_path+'/t3-w3-3.csv')
                df=pd.read_csv(output_path+'/t3-w3-3-2.csv')
                # df=pd.read_csv(output_path+'/eq_curve_'+delta+'.csv')
              


                initial_equity=10000000
                poe=0.01
                equity=poe*initial_equity
                df['lotsize']=1
                df['qty']=1
                df['Exposure']=df['EQ_Close_CE']*2*df['lotsize']
                df['Margin_per_share']=df['EQ_Close_CE']/leverage
                df['Min_premium']=df['Margin_per_share']*.01
                df['Margin']=df['Exposure']/leverage
                df['Total_Premium']=df["OPT_Close_CE"]+df[ "OPT_Close_PE"]
                # df['units'] = (initial_equity/df['Margin']).astype(int)
                # df['qty_final'] = df["qty"]*df['units']

                # print(6,nnn1)
                temp=0
                if nnn==4:
                    for idi, row in df.iterrows():
                        
                        if idi==0:
                            df.loc[idi,'CE_Points']=df.loc[idi,'PE_Points']=df.loc[idi,'Points']=0
                            pl=pl+df.loc[idi,'Points']
                        # if row["Marker"] == "F":
                            df.loc[idi, 'pl_check'] = pl + initial_equity
                            df.loc[idi+1:, 'pl_check'] = np.nan
                            
                            (df.loc[idi ,'available_margin']) = (df.loc[idi ,'pl_check'])*leverage

                            df.loc[idi , 'units_new'] = (((df.loc[idi ,'available_margin']))/df.loc[idi, 'EQ_Close_CE'])
                            df.loc[idi+1: , 'units_new'] = np.nan

                            df.loc[idi,  'qty_final'] = df.loc[idi,"qty"] * df.loc[idi, 'units_new']
                            df.loc[idi+1:, 'qty_final'] = np.nan
                            df['qty_final']=df['qty_final'].ffill()
                            # df.loc[idi, 'plvalue_' + index] = 0
                            # df.loc[idi, 'transaction_cost'] = 0
                            contract_value = df.loc[idi, "qty_final"]*(df.loc[idi,  "OPT_Close_CE"]+df.loc[idi,  "OPT_Close_PE"])
                            continue
                        if row["Marker"] != "F":
                            df.loc[idi,'CE_Points']=-df.loc[idi,  "OPT_Close_CE"]+df.loc[idi-1,  "OPT_Close_CE"]
                            df.loc[idi,'PE_Points']=-df.loc[idi,  "OPT_Close_PE"]+df.loc[idi-1,  "OPT_Close_PE"]
                            df.loc[idi,'Points']=(-df.loc[idi,  "OPT_Close_PE"]+df.loc[idi-1,  "OPT_Close_PE"])+(-df.loc[idi,  "OPT_Close_CE"]+df.loc[idi-1,  "OPT_Close_CE"])
                            # pl=pl+df.loc[idi,'Points']
                            df.loc[idi,'Points_final']=df.loc[idi,'Points']*df.loc[idi,'qty_final']
                            pl=pl+df.loc[idi,'Points_final']
                            
                        if row["Marker"] == "F":
                            df.loc[idi,'CE_Points']=df.loc[idi,'PE_Points']=df.loc[idi,'Points']=0
                            pl=pl+df.loc[idi,'Points']
                        # if row["Marker"] == "F":
                            df.loc[idi, 'pl_check'] = pl + initial_equity
                            df.loc[idi+1:, 'pl_check'] = np.nan
                            
                            (df.loc[idi ,'available_margin']) = (df.loc[idi ,'pl_check'])*leverage

                            df.loc[idi , 'units_new'] = (((df.loc[idi ,'available_margin']))/df.loc[idi, 'EQ_Close_CE'])
                            df.loc[idi+1: , 'units_new'] = np.nan

                            df.loc[idi,  'qty_final'] = df.loc[idi,"qty"] * df.loc[idi, 'units_new']
                            df.loc[idi+1:, 'qty_final'] = np.nan
                            df['qty_final']=df['qty_final'].ffill()
                            
                            df.loc[idi,'Points_final']=df.loc[idi,'Points']*df.loc[idi,'qty_final']
                            pl=pl+df.loc[idi,'Points_final']
                            
                            # df.loc[idi, 'plvalue_' + index] = 0
                            # df.loc[idi, 'transaction_cost'] = 0
                            contract_value = df.loc[idi, "qty_final"]*(df.loc[idi,  "OPT_Close_CE"]+df.loc[idi,  "OPT_Close_PE"])
                        if row['Marker']=='L':
                            contract_value += df.loc[idi, "qty_final"]*(df.loc[idi,  "OPT_Close_CE"]+df.loc[idi,  "OPT_Close_PE"])
                            df.loc[idi,'transaction_cost']=contract_value*0.01
                            df.loc[idi,'Points_final']=df.loc[idi,'Points']*df.loc[idi,'qty_final']-df.loc[idi,'transaction_cost']

                else:       
                    for idi, row in df.iterrows():
                        
                        if idi==0:
                            df.loc[idi,'CE_Points']=df.loc[idi,'PE_Points']=df.loc[idi,'Points']=0
                            pl=pl+df.loc[idi,'Points']
                        # if row["Marker"] == "F":
                            df.loc[idi, 'pl_check'] = (pl + initial_equity)
                            df.loc[idi+1:, 'pl_check'] = np.nan
                            
                            (df.loc[idi ,'available_margin']) = (df.loc[idi ,'pl_check'])*leverage

                            df.loc[idi , 'units_new'] = (((df.loc[idi ,'available_margin']))/df.loc[idi, 'EQ_Close_CE'])
                            df.loc[idi+1: , 'units_new'] = np.nan

                            df.loc[idi,  'qty_final'] = df.loc[idi,"qty"] * df.loc[idi, 'units_new']
                            df.loc[idi+1:, 'qty_final'] = np.nan
                            df['qty_final']=df['qty_final'].ffill()
                            # df.loc[idi, 'plvalue_' + index] = 0
                            # df.loc[idi, 'transaction_cost'] = 0
                            contract_value = df.loc[idi, "qty_final"]*(df.loc[idi,  "OPT_Close_CE"]+df.loc[idi,  "OPT_Close_PE"])
                            continue

                        if row['Marker2']=='L':# and row['Marker']!='L':
                            df.loc[idi,'CE_Points']=0 #-df.loc[idi,  "OPT_Close_CE"]+df.loc[idi-1,  "OPT_Close_CE"]
                            df.loc[idi,'PE_Points']=0 #-df.loc[idi,  "OPT_Close_PE"]+df.loc[idi-1,  "OPT_Close_PE"]
                            df.loc[idi,'Points']=0 #(-df.loc[idi,  "OPT_Close_PE"]+df.loc[idi-1,  "OPT_Close_PE"])+(-df.loc[idi,  "OPT_Close_CE"]+df.loc[idi-1,  "OPT_Close_CE"])

                        else:
                            if nnn>0:
                                # if row['Marker3']=='F' and row['Status1']==1:
                                #     df.loc[idi,'CE_Points']=0 #-df.loc[idi,  "OPT_Close_CE"]+df.loc[idi-1,  "OPT_Close_CE"]
                                #     df.loc[idi,'PE_Points']=0 #-df.loc[idi,  "OPT_Close_PE"]+df.loc[idi-1,  "OPT_Close_PE"]
                                #     df.loc[idi,'Points']=0
                                # else:
                                if row["Marker"] == "F":
                                    df.loc[idi,'CE_Points']=df.loc[idi,'PE_Points']=df.loc[idi,'Points']=0
                                    contract_value= df.loc[idi, "qty_final"]*(df.loc[idi,  "OPT_Close_CE"]+df.loc[idi,  "OPT_Close_PE"])
                                    
                                    
                                # if row["Marker"] == "F":
                                    df.loc[idi, 'pl_check'] = (pl + initial_equity)
                                    df.loc[idi+1:, 'pl_check'] = np.nan

                                    (df.loc[idi ,'available_margin']) = (df.loc[idi ,'pl_check'])*leverage

                                    df.loc[idi , 'units_new'] = (((df.loc[idi ,'available_margin']))/df.loc[idi, 'EQ_Close_CE'])
                                    df.loc[idi+1: , 'units_new'] = np.nan

                                    df.loc[idi,  'qty_final'] = df.loc[idi,"qty"] * df.loc[idi, 'units_new']
                                    df.loc[idi+1:, 'qty_final'] = np.nan
                                    df['qty_final']=df['qty_final'].ffill()
                                    df.loc[idi,'Points_final']=df.loc[idi,'Points']*df.loc[idi,'qty_final']
                                    pl=pl+df.loc[idi,'Points_final']
                            # df.loc[idi, 'plvalue_' + index] = 0
                            # df.loc[idi, 'transaction_cost'] = 0

                                elif df.loc[idi-1,'Status1']==1:
                                    df.loc[idi,'CE_Points']=-df.loc[idi,  "OPT_Close_CE"]+df.loc[idi-1,  "new_OPT_Close_CE"]
                                    df.loc[idi,'PE_Points']=-df.loc[idi,  "OPT_Close_PE"]+df.loc[idi-1,  "new_OPT_Close_PE"]
                                    df.loc[idi,'Points']=df.loc[idi,'CE_Points']+df.loc[idi,'PE_Points']
                                    
                                    contract_value= df.loc[idi, "qty_final"]*(df.loc[idi-1,  "new_OPT_Close_CE"]+df.loc[idi-1,  "new_OPT_Close_PE"])
                                # if row["Marker"] == "F":
                                    df.loc[idi, 'pl_check'] = (pl + initial_equity)
                                    df.loc[idi+1:, 'pl_check'] = np.nan

                                    (df.loc[idi ,'available_margin']) = (df.loc[idi ,'pl_check'])*leverage

                                    df.loc[idi , 'units_new'] = (((df.loc[idi ,'available_margin']))/df.loc[idi, 'EQ_Close_CE'])
                                    df.loc[idi+1: , 'units_new'] = np.nan

                                    df.loc[idi,  'qty_final'] = df.loc[idi,"qty"] * df.loc[idi, 'units_new']
                                    df.loc[idi+1:, 'qty_final'] = np.nan
                                    df['qty_final']=df['qty_final'].ffill()
                                    df.loc[idi,'Points_final']=df.loc[idi,'Points']*df.loc[idi,'qty_final']
                                    pl=pl+df.loc[idi,'Points_final']
                                    
                                else:
                                    df.loc[idi,'CE_Points']=-df.loc[idi,  "OPT_Close_CE"]+df.loc[idi-1,  "OPT_Close_CE"]
                                    df.loc[idi,'PE_Points']=-df.loc[idi,  "OPT_Close_PE"]+df.loc[idi-1,  "OPT_Close_PE"]
                                    df.loc[idi,'Points']=(-df.loc[idi,  "OPT_Close_PE"]+df.loc[idi-1,  "OPT_Close_PE"])+(-df.loc[idi,  "OPT_Close_CE"]+df.loc[idi-1,  "OPT_Close_CE"])
                                    df.loc[idi,'Points_final']=df.loc[idi,'Points']*df.loc[idi,'qty_final']
                                    pl=pl+df.loc[idi,'Points_final']
                                                                        
                                if row['Marker']=='L' or df.loc[idi,'Status1']==1:
                                    contract_value+= df.loc[idi, "qty_final"]*(df.loc[idi,  "OPT_Close_CE"]+df.loc[idi,  "OPT_Close_PE"])
                                    df.loc[idi,'transaction_cost']=contract_value*0.01
                                    pl=pl-df.loc[idi,'transaction_cost']
                                    df.loc[idi,'Points_final']=df.loc[idi,'Points']*df.loc[idi,'qty_final']-df.loc[idi,'transaction_cost']

                                                                
                                                                    
#                                 if row['Marker']=='F' or df.loc[idi,'Status1']==1:
#                                     df.loc[idi,'CE_Points']=df.loc[idi,'PE_Points']=df.loc[idi,'Points']=0
#                                     pl=pl+df.loc[idi,'Points']
#                                 # if row["Marker"] == "F":
#                                     df.loc[idi, 'pl_check'] = pl + initial_equity
#                                     df.loc[idi+1:, 'pl_check'] = np.nan
#                                     # df.loc[idi,'CE_Points']=df.loc[idi,'PE_Points']=df.loc[idi,'Points']=0

#                                     (df.loc[idi ,'available_margin']) = (df.loc[idi ,'pl_check'])

#                                     df.loc[idi , 'units_new'] = (((df.loc[idi ,'pl_check']))/df.loc[idi, 'Margin'])
#                                     df.loc[idi+1: , 'units_new'] = np.nan

#                                     df.loc[idi,  'qty_final'] = df.loc[idi,"qty"] * df.loc[idi, 'units_new']
#                                     df.loc[idi+1:, 'qty_final'] = np.nan
#                                     df['qty_final']=df['qty_final'].ffill()
#                                     # df.loc[idi, 'plvalue_' + index] = 0
#                                     contract_value = df.loc[idi, "qty_final"]*(df.loc[idi,  "OPT_Close_CE"]+df.loc[idi,  "OPT_Close_PE"])
#                                     if df.loc[idi-1,'Status1']==1:
#                                         contract_value = df.loc[idi, "qty_final"]*(df.loc[idi-1,  "new_OPT_Close_CE"]+df.loc[idi-1,  "new_OPT_Close_PE"])
                                

                df['Points_final_2']=df['Points']*df['qty_final']
                
                df['Total_points']=df['Points_final'].cumsum()
                # print(7,nnn1)
                df.to_csv(output_path+'/pl_points_'+delta+'.csv')

                df=pd.read_csv(output_path+'/pl_points_'+delta+'.csv')
                temp=0
                df['Total_points']=df['Points_final'].cumsum()
                df['eq_curve']=df['Points_final']
                df.loc[0,'eq_curve']=initial_equity
                df['eq_curve']=df['eq_curve'].cumsum()
                output_path1=r'C:\Projects\Straddles\NEW\2021-2023\poe-'+str(leverage)+'xlev\Weekly Adj/'+str(nnn)+'/'+str(t)
                if not os.path.exists(output_path1):
                    os.makedirs(output_path1)
                df.to_csv(output_path1+'/eq_curve_'+delta+'_'+str(t)+'.csv')
                print((output_path1+'/eq_curve_'+delta+'_'+str(t)+'.csv'))
                print('Done')
                # xx
                # print(7,nnn1)
                

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/100/eq_curve_50_100.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/25/eq_curve_50_25.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/50/eq_curve_50_50.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/75/eq_curve_50_75.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/125/eq_curve_50_125.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/150/eq_curve_50_150.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/175/eq_curve_50_175.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/200/eq_curve_50_200.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/2/100/eq_curve_50_100.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/2/25/eq_curve_50_25.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/2/50/eq_curve_50_50.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6x

  0%|          | 0/3 [00:00<?, ?it/s]

C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/100/eq_curve_45_100.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/25/eq_curve_45_25.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/50/eq_curve_45_50.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/75/eq_curve_45_75.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/125/eq_curve_45_125.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/150/eq_curve_45_150.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/175/eq_curve_45_175.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/200/eq_curve_45_200.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/2/100/eq_curve_45_100.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/2/25/eq_curve_45_25.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/2/50/eq_curve_45_50.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6x

...

C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/100/eq_curve_25_100.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/25/eq_curve_25_25.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/50/eq_curve_25_50.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/75/eq_curve_25_75.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/125/eq_curve_25_125.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/150/eq_curve_25_150.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/175/eq_curve_25_175.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/200/eq_curve_25_200.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/2/100/eq_curve_25_100.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/2/25/eq_curve_25_25.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/2/50/eq_curve_25_50.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6x

  0%|          | 0/3 [00:00<?, ?it/s]

C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/100/eq_curve_20_100.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/25/eq_curve_20_25.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/50/eq_curve_20_50.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/75/eq_curve_20_75.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/125/eq_curve_20_125.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/150/eq_curve_20_150.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/175/eq_curve_20_175.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/200/eq_curve_20_200.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/2/100/eq_curve_20_100.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/2/25/eq_curve_20_25.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/2/50/eq_curve_20_50.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6x

  0%|          | 0/3 [00:00<?, ?it/s]

C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/100/eq_curve_15_100.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/25/eq_curve_15_25.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/50/eq_curve_15_50.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/75/eq_curve_15_75.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/125/eq_curve_15_125.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/150/eq_curve_15_150.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/175/eq_curve_15_175.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/1/200/eq_curve_15_200.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/2/100/eq_curve_15_100.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/2/25/eq_curve_15_25.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6xlev\Weekly Adj/2/50/eq_curve_15_50.csv
Done
C:\Projects\Straddles\NEW\2021-2023\poe-6x

In [ ]:
####merge curves######
delta_list=['50','45', '40', '35', '30', '25','20','15']
final=pd.DataFrame()
N1=[1,2,3,4]
trigger = [100,25,50,75,125,150, 175, 200]
leverage=6
for delta in tqdm(delta_list):
    # print(delta)
    for nnn in (N1):
        # print('B',nnn)
#         output_path=r'C:\Projects\Straddles\NEW\with_position_sizing\Min_premium\Weekly base case'+'/'+str(nnn)+'/'
        
#         result=pd.read_csv(output_path+'/eq_curve_'+delta+'.csv',usecols=['Date','eq_curve'])
#         if len(final)==0:
#             final['Date']=result['Date']
#         result.rename(columns = {'eq_curve':delta+'_0_W_'+str(nnn)}, inplace = True)
#         final=final.merge(result,on='Date',how='outer')
        if nnn!=0:
            for t in trigger:
                # print('A',nnn)
                output_path=r'C:\Projects\Straddles\NEW\2021-2023\poe-'+str(leverage)+'xlev\Weekly Adj/'+str(nnn)+'/'+str(t)
                if nnn==4 :
                    if t==100:
                    # output_path=r'C:\Projects\Straddles\NEW\N_days'+'/'+str(nnn)+'/'+delta+'_delta'
                        df=pd.read_csv(output_path+'/eq_curve_'+delta+'_'+str(t)+'.csv',usecols=['Date','eq_curve'])
                        if len(final)==0:
                            final['Date']=df['Date']
                        final=final.merge(df[['Date','eq_curve']],on='Date',how='outer')
                        final.rename(columns = {'eq_curve':delta+'_'+str(t)+'_W_'+str(nnn)}, inplace = True)
                else:
                    df=pd.read_csv(output_path+'/eq_curve_'+delta+'_'+str(t)+'.csv',usecols=['Date','eq_curve'])
                    if len(final)==0:
                        final['Date']=df['Date']
                    final=final.merge(df[['Date','eq_curve']],on='Date',how='outer')
                    final.rename(columns = {'eq_curve':delta+'_'+str(t)+'_W_'+str(nnn)}, inplace = True)
# display(final)

final.ffill(inplace=True)
final.bfill(inplace=True)
final.to_csv(r'C:\Projects\Straddles\NEW\2021-2023/W merged curves 6x.csv')
final

In [62]:
####merge curves######

final=pd.DataFrame()
N1=[1,2,3,4]
trigger = [100,25,50,75,125,150, 175, 200]

for delta in tqdm(delta_list):
    # print(delta)
    for nnn in (N1):
        # print('B',nnn)
#         output_path=r'C:\Projects\Straddles\NEW\with_position_sizing\Min_premium\Weekly base case'+'/'+str(nnn)+'/'
        
#         result=pd.read_csv(output_path+'/eq_curve_'+delta+'.csv',usecols=['Date','eq_curve'])
#         if len(final)==0:
#             final['Date']=result['Date']
#         result.rename(columns = {'eq_curve':delta+'_0_W_'+str(nnn)}, inplace = True)
#         final=final.merge(result,on='Date',how='outer')
        if nnn!=0:
            for t in trigger:
                # print('A',nnn)
                output_path=r'C:\Projects\Straddles\NEW\Revised\with_position_sizing\poe-5xlev\Weekly Adj/'+'/'+str(nnn)+'/'+str(t)
                if nnn==4 :
                    if t==100:
                    # output_path=r'C:\Projects\Straddles\NEW\N_days'+'/'+str(nnn)+'/'+delta+'_delta'
                        df=pd.read_csv(output_path+'/eq_curve_'+delta+'_'+str(t)+'.csv',usecols=['Date','eq_curve'])
                        if len(final)==0:
                            final['Date']=df['Date']
                        final=final.merge(df[['Date','eq_curve']],on='Date',how='outer')
                        final.rename(columns = {'eq_curve':delta+'_'+str(t)+'_W_'+str(nnn)}, inplace = True)
                else:
                    df=pd.read_csv(output_path+'/eq_curve_'+delta+'_'+str(t)+'.csv',usecols=['Date','eq_curve'])
                    if len(final)==0:
                        final['Date']=df['Date']
                    final=final.merge(df[['Date','eq_curve']],on='Date',how='outer')
                    final.rename(columns = {'eq_curve':delta+'_'+str(t)+'_W_'+str(nnn)}, inplace = True)
# display(final)

final.ffill(inplace=True)
final.bfill(inplace=True)
final.to_csv(r'C:\Projects\Straddles\NEW\Revised\with_position_sizing\poe-5xlev\Weekly Adj/W merged curves with 4 new.csv')
final

  0%|          | 0/8 [00:00<?, ?it/s]

,Date,50_100_W_1,50_25_W_1,50_50_W_1,50_75_W_1,50_125_W_1,50_150_W_1,50_175_W_1,50_200_W_1,50_100_W_2,50_25_W_2,50_50_W_2,50_75_W_2,50_125_W_2,50_150_W_2,50_175_W_2,50_200_W_2,50_100_W_3,50_25_W_3,50_50_W_3,50_75_W_3,50_125_W_3,50_150_W_3,50_175_W_3,50_200_W_3,50_100_W_4,45_100_W_1,45_25_W_1,45_50_W_1,45_75_W_1,45_125_W_1,45_150_W_1,45_175_W_1,45_200_W_1,45_100_W_2,45_25_W_2,45_50_W_2,45_75_W_2,45_125_W_2,45_150_W_2,45_175_W_2,45_200_W_2,45_100_W_3,45_25_W_3,45_50_W_3,45_75_W_3,45_125_W_3,45_150_W_3,45_175_W_3,45_200_W_3,45_100_W_4,40_100_W_1,40_25_W_1,40_50_W_1,40_75_W_1,40_125_W_1,40_150_W_1,40_175_W_1,40_200_W_1,40_100_W_2,...,25_200_W_2,25_100_W_3,25_25_W_3,25_50_W_3,25_75_W_3,25_125_W_3,25_150_W_3,25_175_W_3,25_200_W_3,25_100_W_4,20_100_W_1,20_25_W_1,20_50_W_1,20_75_W_1,20_125_W_1,20_150_W_1,20_175_W_1,20_200_W_1,20_100_W_2,20_25_W_2,20_50_W_2,20_75_W_2,20_125_W_2,20_150_W_2,20_175_W_2,20_200_W_2,20_100_W_3,20_25_W_3,20_50_W_3,20_75_W_3,20_125_W_3,20_150_W_3,20_175_W_3,20_200_W_3,20_100_W_4,15_100_W_1,15_25_W_1,15_50_W_1,15_75_W_1,15_125_W_1,15_150_W_1,15_175_W_1,15_200_W_1,15_100_W_2,15_25_W_2,15_50_W_2,15_75_W_2,15_125_W_2,15_150_W_2,15_175_W_2,15_200_W_2,15_100_W_3,15_25_W_3,15_50_W_3,15_75_W_3,15_125_W_3,15_150_W_3,15_175_W_3,15_200_W_3,15_100_W_4
0,2016-05-27,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,...,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07
1,2016-05-30,1.019758e+07,1.019758e+07,1.019758e+07,1.019758e+07,1.019758e+07,1.019758e+07,1.019758e+07,1.019758e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.018630e+07,1.018630e+07,1.018630e+07,1.018630e+07,1.018630e+07,1.018630e+07,1.018630e+07,1.018630e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.019730e+07,1.019730e+07,1.019730e+07,1.019730e+07,1.019730e+07,1.019730e+07,1.019730e+07,1.019730e+07,1.000000e+07,...,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.011007e+07,1.011007e+07,1.011007e+07,1.011007e+07,1.011007e+07,1.011007e+07,1.011007e+07,1.011007e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+0

In [63]:
final=pd.DataFrame()
N1=[1]
trigger = [100,25,50,75,125,150, 175, 200]
temp=1
for delta in tqdm(delta_list):
    # print(delta)
    for nnn in (N1):
        # print('B',nnn)
        if nnn==1:
            output_path=r'C:\Projects\Straddles\NEW\Revised\with_position_sizing\poe-5xlev\Monthly base case'+'/'+str(nnn)

            result=pd.read_csv(output_path+'/eq_curve_'+delta+'.csv',usecols=['Date','eq_curve'])
            if len(final)==0:
                output_path1=r'C:\Projects\Straddles\NEW/N_days/'+'/'+str(nnn)+'/'+delta+'_delta'
                df1=pd.read_csv(output_path1+'/eq_curve_'+delta+'.csv',usecols=['Date','eq_curve'])
                final['Date']=pd.merge(result['Date'],df1['Date'],how='outer').sort_values(by='Date').reset_index(drop=True)
            
            result.rename(columns = {'eq_curve':delta+'_0_M_'+str(nnn)}, inplace = True)
            
            final=final.merge(result,on='Date',how='outer')
            final[delta+'_0_M_'+str(nnn)].ffill(inplace=True)
            if nnn!=0:
                for t in trigger:
                    # print('A',nnn)
                    output_path=r'C:\Projects\Straddles\NEW\Revised\with_position_sizing\poe-5xlev\Monthly Adj/'+'/'+str(nnn)+'/'+str(t)
                    if nnn==4:
                        output_path=r'D:\Straddles Backtest Files\NEW\N_days'+'/'+str(nnn)+'/'+delta+'_delta'
                        df=pd.read_csv(output_path+'/eq_curve_'+delta+'.csv',usecols=['Date','eq_curve'])
                    else:
                        df=pd.read_csv(output_path+'/eq_curve_'+delta+'_'+str(t)+'.csv',usecols=['Date','eq_curve'])
                    if len(final)==0:
                        final['Date']=df['Date']
                    final=final.merge(df[['Date','eq_curve']],on='Date',how='outer')
                    final['eq_curve'].ffill(inplace=True)
                    final.rename(columns = {'eq_curve':delta+'_'+str(t)+'_M_'+str(nnn)}, inplace = True)

# display(final)

df=pd.read_csv(r'C:\Projects\Straddles\NEW\Revised\with_position_sizing\poe-5xlev\Weekly Adj/W merged curves with 4 new.csv',parse_dates=['Date'])
df.drop(columns=['Unnamed: 0'],inplace=True)
final['Date']=pd.to_datetime(final['Date'])
newdf=pd.merge(final,df,on='Date',how='outer')#.sort_values(by='Date').reset_index()
newdf

  0%|          | 0/8 [00:00<?, ?it/s]

,Date,50_0_M_1,50_100_M_1,50_25_M_1,50_50_M_1,50_75_M_1,50_125_M_1,50_150_M_1,50_175_M_1,50_200_M_1,45_0_M_1,45_100_M_1,45_25_M_1,45_50_M_1,45_75_M_1,45_125_M_1,45_150_M_1,45_175_M_1,45_200_M_1,40_0_M_1,40_100_M_1,40_25_M_1,40_50_M_1,40_75_M_1,40_125_M_1,40_150_M_1,40_175_M_1,40_200_M_1,35_0_M_1,35_100_M_1,35_25_M_1,35_50_M_1,35_75_M_1,35_125_M_1,35_150_M_1,35_175_M_1,35_200_M_1,30_0_M_1,30_100_M_1,30_25_M_1,30_50_M_1,30_75_M_1,30_125_M_1,30_150_M_1,30_175_M_1,30_200_M_1,25_0_M_1,25_100_M_1,25_25_M_1,25_50_M_1,25_75_M_1,25_125_M_1,25_150_M_1,25_175_M_1,25_200_M_1,20_0_M_1,20_100_M_1,20_25_M_1,20_50_M_1,20_75_M_1,...,25_200_W_2,25_100_W_3,25_25_W_3,25_50_W_3,25_75_W_3,25_125_W_3,25_150_W_3,25_175_W_3,25_200_W_3,25_100_W_4,20_100_W_1,20_25_W_1,20_50_W_1,20_75_W_1,20_125_W_1,20_150_W_1,20_175_W_1,20_200_W_1,20_100_W_2,20_25_W_2,20_50_W_2,20_75_W_2,20_125_W_2,20_150_W_2,20_175_W_2,20_200_W_2,20_100_W_3,20_25_W_3,20_50_W_3,20_75_W_3,20_125_W_3,20_150_W_3,20_175_W_3,20_200_W_3,20_100_W_4,15_100_W_1,15_25_W_1,15_50_W_1,15_75_W_1,15_125_W_1,15_150_W_1,15_175_W_1,15_200_W_1,15_100_W_2,15_25_W_2,15_50_W_2,15_75_W_2,15_125_W_2,15_150_W_2,15_175_W_2,15_200_W_2,15_100_W_3,15_25_W_3,15_50_W_3,15_75_W_3,15_125_W_3,15_150_W_3,15_175_W_3,15_200_W_3,15_100_W_4
0,2016-01-01,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-01-04,1.027152e+07,1.027152e+07,1.023114e+07,1.023114e+07,1.027152e+07,1.027152e+07,1.027152e+07,1.027152e+07,1.027152e+07,1.039011e+07,1.039011e+07,1.035442e+07,1.035442e+07,1.035442e+07,1.039011e+07,1.039011e+07,1.039011e+07,1.039011e+07,1.040229e+07,1.040229e+07,1.037151e+07,1.037151e+07,1.040229e+07,1.040229e+07,1.040229e+07,1.040229e+07,1.040229e+07,1.041550e+07,1.041550e+07,1.039073e+07,1.039073e+07,1.041550e+07,1.041550e+07,1.041550e+07,1.041550e+07,1.041550e+07,1.034578e+07,1.034578e+07,1.032346e+07,1.034578e+07,1.034578e+07,1.034578e+07,1.034578e+07,1.034578e+07,1.034578e+07,1.030146e+07,1.030146e+07,1.030146e+07,1.030146e+07,1.030146e+07,1.030146e+07,1.030146e+07,1.030146e+07,1.030146e+07,1.043487e+07,1.043487e+07,1.043487e+07,1.043487e+07,1.043487e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-01-05,1.025493e+07,1.025493e+07,1.014410e+07,1.014410e+07,1.025493e+07,1.025493e+07,1.025493e+07,1.025493e+07,1.025493e+07,1.038893e+07,1.038893e+07,1.028542e+07,1.028542e+07,1.028542e+07,1.038893e+07,1.038893e+07,1.038893e+07,1.038893e+07,1.037983e+07,1.037983e+07,1.029801e+07,1.029801e+07,1.034928e+07,1.037983e+07,1.037983e+07,1.037983e+07,1.037983e+07,1.043839e+07,1.043839e+07,1.031945e+07,1.031945e+07,1.043839e+07,1.043839e+07,1.043839e+07,1.043839e+07,1.043839e+07,1.037631e+07,1.037631e+07,1.026575e+07,1.037631e+07,1.037631e+07,1.037631e+07,1.037631e+07,1.037631e+07,1.037631e+07,1.029911e+07,1.029911e+07,1.029911e+07,1.029911e+07,1.029911e+07,1.029911e+07,1.029911e+07,

In [64]:
newdf.to_csv(r'C:\Projects\Straddles\NEW\Revised\with_position_sizing\poe-5xlev/merged curves with w4 new.csv')


In [65]:
newdf['Date']=pd.to_datetime(newdf['Date'])
newdf1=newdf[newdf['Date']<=pd.to_datetime('22-07-2021')].reset_index(drop=True)
newdf1=newdf1[newdf1['Date']>=pd.to_datetime('30-05-2016')].reset_index(drop=True)
newdf1.ffill(inplace=True)
newdf1.to_csv(r'C:\Projects\Straddles\NEW\Revised\with_position_sizing\poe-5xlev/merged curves with w4 and date filter new.csv')
newdf1

,Date,50_0_M_1,50_100_M_1,50_25_M_1,50_50_M_1,50_75_M_1,50_125_M_1,50_150_M_1,50_175_M_1,50_200_M_1,45_0_M_1,45_100_M_1,45_25_M_1,45_50_M_1,45_75_M_1,45_125_M_1,45_150_M_1,45_175_M_1,45_200_M_1,40_0_M_1,40_100_M_1,40_25_M_1,40_50_M_1,40_75_M_1,40_125_M_1,40_150_M_1,40_175_M_1,40_200_M_1,35_0_M_1,35_100_M_1,35_25_M_1,35_50_M_1,35_75_M_1,35_125_M_1,35_150_M_1,35_175_M_1,35_200_M_1,30_0_M_1,30_100_M_1,30_25_M_1,30_50_M_1,30_75_M_1,30_125_M_1,30_150_M_1,30_175_M_1,30_200_M_1,25_0_M_1,25_100_M_1,25_25_M_1,25_50_M_1,25_75_M_1,25_125_M_1,25_150_M_1,25_175_M_1,25_200_M_1,20_0_M_1,20_100_M_1,20_25_M_1,20_50_M_1,20_75_M_1,...,25_200_W_2,25_100_W_3,25_25_W_3,25_50_W_3,25_75_W_3,25_125_W_3,25_150_W_3,25_175_W_3,25_200_W_3,25_100_W_4,20_100_W_1,20_25_W_1,20_50_W_1,20_75_W_1,20_125_W_1,20_150_W_1,20_175_W_1,20_200_W_1,20_100_W_2,20_25_W_2,20_50_W_2,20_75_W_2,20_125_W_2,20_150_W_2,20_175_W_2,20_200_W_2,20_100_W_3,20_25_W_3,20_50_W_3,20_75_W_3,20_125_W_3,20_150_W_3,20_175_W_3,20_200_W_3,20_100_W_4,15_100_W_1,15_25_W_1,15_50_W_1,15_75_W_1,15_125_W_1,15_150_W_1,15_175_W_1,15_200_W_1,15_100_W_2,15_25_W_2,15_50_W_2,15_75_W_2,15_125_W_2,15_150_W_2,15_175_W_2,15_200_W_2,15_100_W_3,15_25_W_3,15_50_W_3,15_75_W_3,15_125_W_3,15_150_W_3,15_175_W_3,15_200_W_3,15_100_W_4
0,2016-05-30,2.293726e+07,1.027178e+07,1.033206e+07,1.096493e+07,1.215395e+07,1.211276e+07,1.448811e+07,1.277496e+07,1.321067e+07,2.225265e+07,1.057183e+07,1.058971e+07,1.117842e+07,1.084687e+07,1.263968e+07,1.260717e+07,1.308185e+07,1.437978e+07,2.194013e+07,1.094558e+07,1.070779e+07,1.309827e+07,1.334920e+07,1.202856e+07,1.298061e+07,1.408178e+07,1.347207e+07,2.047121e+07,9.938575e+06,1.134645e+07,1.093293e+07,8.838425e+06,1.083589e+07,1.139814e+07,1.145621e+07,1.256830e+07,2.016756e+07,1.105920e+07,1.036209e+07,8.606401e+06,9.714390e+06,1.117064e+07,1.106899e+07,1.080549e+07,1.114926e+07,2.040164e+07,1.134169e+07,9.327398e+06,1.061289e+07,1.124429e+07,1.142393e+07,1.142393e+07,1.124464e+07,1.091034e+07,1.866786e+07,1.096739e+07,9.192842e+06,1.011755e+07,1.039926e+07,...,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.011007e+07,1.011007e+07,1.011007e+07,1.011007e+07,1.011007e+07,1.011007e+07,1.011007e+07,1.011007e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.009294e+07,1.009294e+07,1.009294e+07,1.009294e+07,1.009294e+07,1.009294e+07,1.009294e+07,1.009294e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07
1,2016-05-31,2.286354e+07,1.023790e+07,1.029798e+07,1.092876e+07,1.211387e+07,1.207281e+07,1.444033e+07,1.273283e+07,1.316710e+07,2.210942e+07,1.050220e+07,1.051996e+07,1.110480e+07,1.077542e+07,1.255643e+07,1.252413e+07,1.299569e+07,1.428507e+07,2.184023e+07,1.089470e+07,1.065801e+07,1.303738e+07,1.328714e+07,1.197265e+07,1.292027e+07,1.401632e+07,1.340945e+07,2.036571e+07,9.886399e+06,1.128688e+07,1.087554e+07,8.792025e+06,1.077900e+07,1.133831e+07,1.139607e+07,1.250231e+07,2.006402e+07,1.100153e+07,1.030806e+07,8.561525e+06,9.663737e+06,1.111239e+07,1.101127e+07,1.074915e+07,1.109113e+07,2.029669e+07,1.128256e+07,9.278762e+06,1.055755e+07,1.118566e+07,1.136437e+07,1.136437e+07,1.118601e+07,1.085345e+07,1.862137e+07,1.093975e+07,9.169679e+06,1.009206e+07,1.037306e+07,...,1.013370e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.015675e+07,1.015675e+07,1.015675e+07,1.015675e+07,1.015675e+07,1.015675e+07,1.015675e+07,1.015675e+07,1.010159e+07,1.010159e+07,1.010159e+07,1.010159e+07,1.010159e+07,1.010159e+07,1.010159e+07,1.010159e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07